 Load Libraries

In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

DB Config

In [ ]:
#####DB Config
host= "DevEdgeV32"
port=27017

Set Parameters

In [ ]:
### set parameters
preprocessor_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/preprocessor/'
preprocessor_name='preprocessor.p'
model_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/model/'
model_name= 'model.h5'
selected_Columns=  ['t_bett','t_motor', 't_spindle','M8', 'M121', 'M127', 'M7']# So must be the input of the model
output_variable= 'welle_z'
prediction_variable= 'prediction'
window=10
shift= 1

Funtion to remove t_bett from t_spindel and t_motor

In [ ]:
def remove_t_bett(data: np.ndarray):
    result= data[:, 1:]
    result[:,0]= result[:,0] - data[:,0]
    result[:,1]= result[:,1] - data[:,0]
    return result

Function to apply rounding on the data

In [ ]:
def rounding(signals: np.ndarray):
    return np.round(signals,2)

Function to smooth a signal

In [ ]:
def remove_peeks(dlf: pd.DataFrame, feature: str,window):
    df= dlf.copy()
    new_Feature_name= 'smoothed_' + feature
    df[new_Feature_name]= df[feature].rolling(window).mean().to_list()
    shape= df.shape
    offset= int(np.round(window/2))
    for i in range(df.shape[0] - offset):
        df.loc[i, new_Feature_name]= df.iloc[i + offset][new_Feature_name]
    df= df[df[new_Feature_name] == df[new_Feature_name]]
    return df

Function to create a data set, where each data point is a window of n steps (look window feature)

In [ ]:
def generateDataSource(signal_input= None, output_length: int=1, signal_output= None, window= 1 , shift= 1):
    Signal_Length = signal_input.shape[0]
    num_samples= int((Signal_Length - window +1 ) / shift)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y= np.zeros(shape=(num_samples, output_length, 1))
    for i in range (num_samples):
        x[i]= signal_input[i * shift : i * shift + window]
        if signal_output is not None:
            y[i]= signal_output[i * shift + window - 1 : i * shift + window - 1+ output_length]
    return x, y

Set the time period to get the data

In [ ]:
#start= datetime.strptime("2022-08-30T15:26:06.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") 
start= datetime.strptime("2022-09-24T18:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime("2022-09-26T05:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") 

Read data from the mongo_DB database and apply preprocessing

In [ ]:
## Read raws and preprocessing data
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db.modelLogs.find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
last_prediction= -500
### Go through events in DB
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
        if record['date']< start or record['date'] > end:
            continue
        if 'given2model' not in record:
            if record[prediction_variable] == last_prediction:
                record['given2model']= False
            else:
                last_prediction= record[prediction_variable]
                record['given2model']= True
        keys= record['raw_data'].keys()
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
        #print ('before',record)
        record['raw_data']['date']=[record['date'] + timedelta(hours= +2)]
        record['raw_data'][output_variable]=[record[output_variable]]
        record['raw_data'][prediction_variable]=[record[prediction_variable]]
        record['raw_data']['DRZ2'][0]= float(record['raw_data']['DRZ2'][0])
        # record['raw_data']['M8']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M121']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M127']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M7']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0] - record['raw_data']['t_bett'][0], decimals=2)
        # record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]) - record['raw_data']['t_bett'][0], decimals= 2)
        record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0], decimals=2)
        record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]), decimals= 2)
        #####Add preprocessed Data##########
        record['raw_data']['0']= record['preprocessed_data'][0]
        record['raw_data']['1']= record['preprocessed_data'][1]
        record['raw_data']['2']= record['preprocessed_data'][2]
        record['raw_data']['3']= record['preprocessed_data'][3]
        record['raw_data']['4']= record['preprocessed_data'][4]
        record['raw_data']['5']= record['preprocessed_data'][5]
        ####################################
        record['raw_data']['given2model']= record['given2model']
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
signals['t_motor']= signals['t_motor'].apply(np.float32)
# signals['prediction_abs_error']= signals[prediction_variable] -signals[output_variable]
# signals['prediction_abs_error']= 1000 * signals['prediction_abs_error'].abs()
signals.reset_index(inplace= True)
####To save as csv_File####
# 
# #signals = remove_peeks(signals, output_variable, 4)
output=  signals[output_variable]
machine_prediction= signals[prediction_variable]
rough_signals= signals[selected_Columns].to_numpy()
print(rough_signals.shape)
#signals.rename(columns={'t_spindle': 't_spindel', 'DRZ': 'drhz'}, inplace= True)
print (signals.head(3))
print (signals.tail(10))

Go through the records in database an correct the delay of the welle_z, where a new value of 'welle_z' is available when 'given2model' is true 

In [ ]:
#signals['welle_z']= signals['welle_z'].shift(-1)
length= len(signals)
start= -1
end= -1
phases=[]
for index, row in signals.iterrows():
    if start<0:
        start= index
        continue
    if row['given2model'] or index == length - 1:
        end= index-1
        new_phase= (start, end)
        phases.append(new_phase)
        start= index
        print(new_phase)
for phase in phases:
    start_idx= phase[0]
    end_idx= phase[1]
    signals['welle_z'][start_idx: end_idx + 1]= signals['welle_z'][end_idx+1]

In [ ]:
signals.to_csv('corrected_example.csv')

Visualization of the prediction results and created before during the execution of the experiment

In [ ]:
from turtle import color
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['t_bett','t_motor', 't_spindle' ,'M8', 'M121', 'M127', 'M7', prediction_variable]#'0','1','2','3','4','5'
y_axis_names= selected_Columns
#print(preds)
prediction_error= 1000*(signals[output_variable] - signals[prediction_variable])
fig= make_subplots(rows=len(selected_Columns)+1,cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_Columns, vertical_spacing=0.02)

for i in range(len(selected_Columns)):
    fig.add_trace(go.Scatter(x= signals['date'], y= signals[selected_Columns[i]], name=selected_Columns[i], mode= scatter_mode), row= i+1, col= 1)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
##Draw the prediciton and the real values of displacement on Welle
fig.add_trace(go.Scatter(x= signals['date'],y= signals[output_variable], name=output_variable, mode= scatter_mode),  row= len(selected_Columns) , col= 1)
fig.update_yaxes(title_text= output_variable, row= len(selected_Columns), col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y=   prediction_error, name=' Rest Fehler Micro-Meter', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
## Draw the tolerence +-5
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(prediction_error,5), name='+5 Obere Grenze', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(prediction_error,-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'Prediction Results')